## Preamble

In [1]:
# Load packages
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
path = 'C:/Users/angel/Documents/Economics/Research/Banking Project/data/intermediate/call_reports_wrds/'

# set path as working directory:
os.chdir(path)

In [3]:
# Load legacy data
rcfd_legacy = pd.read_csv(path+'/legacy/RCFD_legacy.csv')
#rcfn_legacy = pd.read_csv(path+'/legacy/RCFN_legacy.csv')
rcon_legacy = pd.read_csv(path+'/legacy/RCON_legacy.csv')
riad_legacy = pd.read_csv(path+'/legacy/RIAD_legacy.csv')

In [4]:
rcfd_legacy.rename(columns={'RSSD9999':'Date', 'RSSD9010': 'Financial Institution Name', 
                     'RSSD9001': 'IDRSSD'}, inplace=True)

riad_legacy.rename(columns={'RSSD9999':'Date', 'RSSD9010': 'Financial Institution Name', 
                     'RSSD9001': 'IDRSSD'}, inplace=True)

rcon_legacy.rename(columns={'RSSD9999':'Date', 'RSSD9010': 'Financial Institution Name', 
                     'RSSD9001': 'IDRSSD'}, inplace=True)

In [5]:
rcfd_legacy['Date'] = pd.to_datetime(rcfd_legacy['Date'], format='%Y%m%d')
riad_legacy['Date'] = pd.to_datetime(riad_legacy['Date'], format='%Y%m%d')
rcon_legacy['Date'] = pd.to_datetime(rcon_legacy['Date'], format='%Y%m%d')

In [6]:
# Load newer data
rcfd = pd.read_csv(path+'/newer/RCFD.csv')
rcfa = pd.read_csv(path+'/newer/RCFA.csv')
rcfn = pd.read_csv(path+'/newer/RCFN.csv')
rcon = pd.read_csv(path+'/newer/RCON.csv')
riad = pd.read_csv(path+'/newer/RIAD.csv')

In [7]:
# Rename column RSSD9999 to Date:
rcfd.rename(columns={'RSSD9999':'Date', 'RSSD9017': 'Financial Institution Name', 
                     'RSSD9001': 'IDRSSD'}, inplace=True)

rcfa.rename(columns={'RSSD9999':'Date', 'RSSD9017': 'Financial Institution Name', 
                     'RSSD9001': 'IDRSSD'}, inplace=True)

rcfn.rename(columns={'RSSD9999':'Date', 'RSSD9017': 'Financial Institution Name', 
                     'RSSD9001': 'IDRSSD'}, inplace=True)

rcon.rename(columns={'RSSD9999':'Date', 'RSSD9017': 'Financial Institution Name', 
                     'RSSD9001': 'IDRSSD'}, inplace=True)

riad.rename(columns={'RSSD9999':'Date', 'RSSD9010': 'Financial Institution Name', 
                     'RSSD9001': 'IDRSSD'}, inplace=True)

In [8]:
riad['Date'] = pd.to_datetime(riad['Date'], format='%Y-%m-%d')
rcfd['Date'] = pd.to_datetime(rcfd['Date'], format='%Y-%m-%d')
rcfa['Date'] = pd.to_datetime(rcfa['Date'], format='%Y-%m-%d')
rcfn['Date'] = pd.to_datetime(rcfn['Date'], format='%Y-%m-%d')
rcon['Date'] = pd.to_datetime(rcon['Date'], format='%Y-%m-%d')

In [28]:
# merge rcon_legacy and rcfd_legacy in 'Date', 'IDRSSD', 'Financial Institution Name':
legacy_df = pd.merge(rcfd_legacy, rcon_legacy, on=['Date', 'IDRSSD', 'Financial Institution Name'], how='outer')

# merge rcon and rcfd in 'Date', 'IDRSSD', 'Financial Institution Name':
newer_df = pd.merge(rcfd, rcon, on=['Date', 'IDRSSD', 'Financial Institution Name'], how='outer')

## Testing Assets

In [36]:
# Compute the difference between the 
legacy_df['Difference Assets'] = legacy_df['RCFD2170'] - legacy_df['RCON2170'] 
legacy_df['Difference Assets (%)'] = legacy_df['Difference Assets']/legacy_df['RCFD2170']
legacy_df[(legacy_df['IDRSSD']==480228) & (legacy_df['Date'].dt.year>2008)
          & (legacy_df['Date'].dt.year<2016)][['Date', 'Financial Institution Name', 'RCFD2170', 'RCON2170', 'Difference Assets', 'Difference Assets (%)']]

,Date,Financial Institution Name,RCFD2170,RCON2170,Difference Assets,Difference Assets (%)
1135005,2009-03-31,BANK OF AMER NA,1.432862e+09,1.353986e+09,78875838.0,0.055048
1142795,2009-06-30,BANK OF AMER NA,1.449558e+09,1.374817e+09,74741495.0,0.051562
1150536,2009-09-30,BANK OF AMER NA,1.460147e+09,1.383597e+09,76550145.0,0.052426
1158181,2009-12-31,BANK OF AMER NA,1.465221e+09,1.389723e+09,75498876.0,0.051527
1165779,2010-03-31,BANK OF AMER NA,1.496322e+09,1.423296e+09,73026133.0,0.048804
1173268,2010-06-30,BANK OF AMER NA,1.518958e+09,1.445545e+09,73412425.0,0.048331
1180674,2010-09-30,BANK OF AMER NA,1.489198e+09,1.404154e+09,85043852.0,0.057107
1188010,2010-12-31,BANK OF AMER NA,1.482278e+09,1.384825e+09,97453617.0,0.065746
1195275,2011-03-31,BANK OF AMER NA,1.451387e+09,1.344516e+09,106870576.0,0.073633
1202453,2011-06-30,BANK OF AMER NA,1.456638e+09,1.344385e+09,112252581.0,0.077063


In [37]:
# do the same for newer data:
newer_df['Difference Assets'] = newer_df['RCFD2170'] - newer_df['RCON2170']
newer_df['Difference Assets (%)'] = newer_df['Difference Assets']/newer_df['RCFD2170']
newer_df[(newer_df['IDRSSD']==480228) & (newer_df['Date'].dt.year>2008)
          & (newer_df['Date'].dt.year<2016)][['Date', 'Financial Institution Name', 'RCFD2170', 'RCON2170', 'Difference Assets', 'Difference Assets (%)']]

,Date,Financial Institution Name,RCFD2170,RCON2170,Difference Assets,Difference Assets (%)
262547,2009-03-31,"BANK OF AMERICA, NATIONAL ASSOCIATION",1.432862e+09,1.353986e+09,78875838.0,0.055048
270053,2009-06-30,"BANK OF AMERICA, NATIONAL ASSOCIATION",1.449558e+09,1.374817e+09,74741495.0,0.051562
277510,2009-09-30,"BANK OF AMERICA, NATIONAL ASSOCIATION",1.460147e+09,1.383597e+09,76550145.0,0.052426
284875,2009-12-31,"BANK OF AMERICA, NATIONAL ASSOCIATION",1.465221e+09,1.389723e+09,75498876.0,0.051527
292181,2010-03-31,"BANK OF AMERICA, NATIONAL ASSOCIATION",1.496322e+09,1.423296e+09,73026133.0,0.048804
299397,2010-06-30,"BANK OF AMERICA, NATIONAL ASSOCIATION",1.518958e+09,1.445545e+09,73412425.0,0.048331
306531,2010-09-30,"BANK OF AMERICA, NATIONAL ASSOCIATION",1.489198e+09,1.404154e+09,85043852.0,0.057107
313594,2010-12-31,"BANK OF AMERICA, NATIONAL ASSOCIATION",1.482278e+09,1.384825e+09,97453617.0,0.065746
320574,2011-03-31,"BANK OF AMERICA, NATIONAL ASSOCIATION",1.451387e+09,NaN,NaN,NaN
327481,2011-06-30,"BANK OF AMERICA, NATIONAL ASSOCIATION",1.456638e+09,NaN,NaN,NaN


## Testing Loans

In [ ]:
# Compute the difference
legacy_df['Difference Loans'] = legacy_df['RCFD1400'] - legacy_df['RCON1400'] 
legacy_df['Difference Loans (%)'] = legacy_df['Difference Loans']/legacy_df['RCFD1400']
legacy_df[(legacy_df['IDRSSD']==480228) & (legacy_df['Date'].dt.year>2008)
          & (legacy_df['Date'].dt.year<2016)][['Date', 'Financial Institution Name', 'RCFD1400', 'RCON1400', 'Difference Loans', 'Difference Loans (%)']]

,Date,Financial Institution Name,RCFD1400,RCON1400,Difference Loans,Difference Loans (%)
1135005,2009-03-31,BANK OF AMER NA,679363673.0,635023851.0,44339822.0,0.065267
1142795,2009-06-30,BANK OF AMER NA,748343238.0,704846390.0,43496848.0,0.058124
1150536,2009-09-30,BANK OF AMER NA,739580188.0,698238925.0,41341263.0,0.055898
1158181,2009-12-31,BANK OF AMER NA,755800605.0,715840657.0,39959948.0,0.052871
1165779,2010-03-31,BANK OF AMER NA,741621407.0,700807106.0,40814301.0,0.055034
1173268,2010-06-30,BANK OF AMER NA,739369114.0,697839169.0,41529945.0,0.056169
1180674,2010-09-30,BANK OF AMER NA,724632971.0,676685295.0,47947676.0,0.066168
1188010,2010-12-31,BANK OF AMER NA,738173018.0,689191850.0,48981168.0,0.066355
1195275,2011-03-31,BANK OF AMER NA,NaN,NaN,NaN,NaN
1202453,2011-06-30,BANK OF AMER NA,NaN,NaN,NaN,NaN


In [39]:
# Compute the difference between the 
newer_df['Difference Loans'] = newer_df['RCFD2122'] - newer_df['RCON2122'] 
newer_df['Difference Loans (%)'] = newer_df['Difference Loans']/newer_df['RCFD2122']
newer_df[(newer_df['IDRSSD']==480228) & (newer_df['Date'].dt.year>2008)
          & (newer_df['Date'].dt.year<2016)][['Date', 'Financial Institution Name', 'RCFD2122', 'RCON2122', 'Difference Loans', 'Difference Loans (%)']]

,Date,Financial Institution Name,RCFD2122,RCON2122,Difference Loans,Difference Loans (%)
262547,2009-03-31,"BANK OF AMERICA, NATIONAL ASSOCIATION",679363673.0,635023851,44339822.0,0.065267
270053,2009-06-30,"BANK OF AMERICA, NATIONAL ASSOCIATION",748343238.0,704846390,43496848.0,0.058124
277510,2009-09-30,"BANK OF AMERICA, NATIONAL ASSOCIATION",739580188.0,698238925,41341263.0,0.055898
284875,2009-12-31,"BANK OF AMERICA, NATIONAL ASSOCIATION",755800605.0,715840657,39959948.0,0.052871
292181,2010-03-31,"BANK OF AMERICA, NATIONAL ASSOCIATION",741621407.0,700807106,40814301.0,0.055034
299397,2010-06-30,"BANK OF AMERICA, NATIONAL ASSOCIATION",739369114.0,697839169,41529945.0,0.056169
306531,2010-09-30,"BANK OF AMERICA, NATIONAL ASSOCIATION",724632971.0,676685295,47947676.0,0.066168
313594,2010-12-31,"BANK OF AMERICA, NATIONAL ASSOCIATION",738173018.0,689191850,48981168.0,0.066355
320574,2011-03-31,"BANK OF AMERICA, NATIONAL ASSOCIATION",742994548.0,688466919,54527629.0,0.073389
327481,2011-06-30,"BANK OF AMERICA, NATIONAL ASSOCIATION",752218997.0,690210855,62008142.0,0.082434


## Testing Deposits

In [41]:
# Compute the difference
legacy_df['Difference Deposits'] = legacy_df['RCFD2200'] - legacy_df['RCON2200'] 
legacy_df['Difference Deposits (%)'] = legacy_df['Difference Deposits']/legacy_df['RCFD1400']
legacy_df[(legacy_df['IDRSSD']==480228) & (legacy_df['Date'].dt.year>2008)
          & (legacy_df['Date'].dt.year<2016)][['Date', 'Financial Institution Name', 'RCFD2200', 'RCON2200', 'Difference Deposits', 'Difference Deposits (%)']]

,Date,Financial Institution Name,RCFD2200,RCON2200,Difference Deposits,Difference Deposits (%)
1135005,2009-03-31,BANK OF AMER NA,9.469968e+08,7.654753e+08,181521484.0,0.267193
1142795,2009-06-30,BANK OF AMER NA,1.008386e+09,8.179893e+08,190396785.0,0.254424
1150536,2009-09-30,BANK OF AMER NA,1.002709e+09,8.071775e+08,195531469.0,0.264382
1158181,2009-12-31,BANK OF AMER NA,1.002692e+09,8.313977e+08,171294659.0,0.226640
1165779,2010-03-31,BANK OF AMER NA,1.000639e+09,8.241690e+08,176469967.0,0.237952
1173268,2010-06-30,BANK OF AMER NA,1.006992e+09,8.289283e+08,178064141.0,0.240833
1180674,2010-09-30,BANK OF AMER NA,1.021724e+09,8.205038e+08,201220188.0,0.277686
1188010,2010-12-31,BANK OF AMER NA,1.038171e+09,8.375558e+08,200614834.0,0.271772
1195275,2011-03-31,BANK OF AMER NA,NaN,8.489263e+08,NaN,NaN
1202453,2011-06-30,BANK OF AMER NA,NaN,9.486802e+08,NaN,NaN


In [44]:
newer_df2 = pd.merge(newer_df, rcfn, on=['Date', 'IDRSSD', 'Financial Institution Name'], how='outer')

In [49]:
# do the same for newer data:
newer_df2['Total Deposits'] = newer_df2['RCON2200']+newer_df2['RCFN2200']
newer_df2['Difference Deposits (%)'] = newer_df2['RCFN2200'] / newer_df2['Total Deposits']
newer_df2[(newer_df2['IDRSSD']==480228) & (newer_df2['Date'].dt.year>2008)
          & (newer_df2['Date'].dt.year<2016)][['Date', 'Financial Institution Name', 'Total Deposits', 'RCON2200', 'RCFN2200', 'Difference Deposits (%)']]

,Date,Financial Institution Name,Total Deposits,RCON2200,RCFN2200,Difference Deposits (%)
262547,2009-03-31,"BANK OF AMERICA, NATIONAL ASSOCIATION",9.469968e+08,765475336,181521484.0,0.191681
270053,2009-06-30,"BANK OF AMERICA, NATIONAL ASSOCIATION",1.008386e+09,817989321,190396785.0,0.188813
277510,2009-09-30,"BANK OF AMERICA, NATIONAL ASSOCIATION",1.002709e+09,807177514,195531469.0,0.195003
284875,2009-12-31,"BANK OF AMERICA, NATIONAL ASSOCIATION",1.002692e+09,831397678,171294659.0,0.170835
292181,2010-03-31,"BANK OF AMERICA, NATIONAL ASSOCIATION",1.000639e+09,824168960,176469967.0,0.176357
299397,2010-06-30,"BANK OF AMERICA, NATIONAL ASSOCIATION",1.006992e+09,828928319,178064141.0,0.176828
306531,2010-09-30,"BANK OF AMERICA, NATIONAL ASSOCIATION",1.021724e+09,820503805,201220188.0,0.196942
313594,2010-12-31,"BANK OF AMERICA, NATIONAL ASSOCIATION",1.038171e+09,837555815,200614834.0,0.193239
320574,2011-03-31,"BANK OF AMERICA, NATIONAL ASSOCIATION",1.047013e+09,848926338,198086971.0,0.189192
327481,2011-06-30,"BANK OF AMERICA, NATIONAL ASSOCIATION",1.060482e+09,948680150,111802314.0,0.105426


## Testing FedFunds Purchased

In [ ]:
rcfd[(rcfd['IDRSSD']==480228) & (rcfd['Date'].dt.year<2002)][['Date', 'Financial Institution Name', 'RCFD2800']]

In [ ]:
rcfd[(rcfd['IDRSSD']==480228) & (rcfd['Date'].dt.year>2001)][['Date', 'Financial Institution Name', 'RCFDB993', 'RCFDB995']]

In [ ]:
rcfd[(rcfd['IDRSSD']==802866) ][['Date', 'Financial Institution Name', 'RCFDB993', 'RCFDB995']]

## Testing Loans Non-Accrual

In [ ]:
rcfd[(rcfd['IDRSSD']==480228)][['Date', 'Financial Institution Name', 'RCFD1403']]

In [ ]:
rcfd[(rcfd['IDRSSD']==802866)][['Date', 'Financial Institution Name', 'RCFD1403']]

## Test Loans Past Due 90 Days

In [ ]:
rcfd[(rcfd['IDRSSD']==802866)][['Date', 'Financial Institution Name', 'RCFD1407']]

In [ ]:
rcfd[(rcfd['IDRSSD']==480228)][['Date', 'Financial Institution Name', 'RCFD1407']]

## Testing Tier 1 Capital

In [ ]:
rcfd[(rcfd['IDRSSD']==480228) & (rcfd['Date'].dt.year>1995) 
                              & (rcfd['Date'].dt.year<2015)][['Date', 'Financial Institution Name','RCFD8274']]

In [ ]:
rcfa[(rcfa['IDRSSD']==480228) & (rcfa['Date'].dt.year>2013)][['Date', 'Financial Institution Name','RCFA8274']]

In [ ]:
rcfd[(rcfd['IDRSSD']==802866) & (rcfd['Date'].dt.year>1995) 
                              & (rcfd['Date'].dt.year<2016)][['Date', 'Financial Institution Name','RCFD8274']]

In [ ]:
rcfa[(rcfa['IDRSSD']==802866) & (rcfa['Date'].dt.year>2013)][['Date', 'Financial Institution Name','RCFA8274']]

## Testing Risk-Weighted Assets

In [ ]:
rcfd[(rcfd['IDRSSD']==480228) & (rcfd['Date'].dt.year>1995) 
                              & (rcfd['Date'].dt.year<2016)][['Date', 'Financial Institution Name','RCFDA223']]

In [ ]:
rcfa[(rcfa['IDRSSD']==480228) & (rcfa['Date'].dt.year>2013)][['Date', 'Financial Institution Name','RCFAA223']]

In [ ]:
rcfd[(rcfd['IDRSSD']==802866) & (rcfd['Date'].dt.year>1995) 
                              & (rcfd['Date'].dt.year<2016)][['Date', 'Financial Institution Name','RCFDA223']]

In [ ]:
rcfa[(rcfa['IDRSSD']==802866) & (rcfa['Date'].dt.year>2013)][['Date', 'Financial Institution Name','RCFAA223']]

## Testing Other Borrowings

In [ ]:
rcfd[(rcfd['IDRSSD']==480228) & (rcfd['Date'].dt.year>1984) 
                              & (rcfd['Date'].dt.year<2002)][['Date', 'Financial Institution Name','RCFD2835', 'RCFD3190']]

In [ ]:
rcfd[(rcfd['IDRSSD']==480228) & (rcfd['Date'].dt.year>2002)][['Date', 'Financial Institution Name','RCFD2835', 'RCFD3190']]

## Testing Cash

In [ ]:
rcfd[(rcfd['IDRSSD']==480228)][['Date', 'Financial Institution Name','RCFD0010']]

In [ ]:
rcfd[(rcfd['IDRSSD']==802866)][['Date', 'Financial Institution Name','RCFD0010']]

## Testing Federal Funds Sold

In [ ]:
rcfd[(rcfd['IDRSSD']==480228) & (rcfd['Date'].dt.year>1983) 
                              & (rcfd['Date'].dt.year<2003)][['Date', 'Financial Institution Name','RCFD1350']]

In [ ]:
rcfd[(rcfd['IDRSSD']==480228) & (rcfd['Date'].dt.year>2001)][['Date', 'Financial Institution Name','RCFDB987', 'RCFDB989']]

## Dean's Variables

In [ ]:
# concatenate the two dataframes:
df = pd.merge(riad, rcfd, on=['Date', 'IDRSSD', 'Financial Institution Name'], how='outer')
df = pd.merge(df, rcfa, on=['Date', 'IDRSSD', 'Financial Institution Name'], how='outer')

In [ ]:
# create a 'Year' column:
df['Year'] = df['Date'].dt.year

# create a 'Quarter' column:
df['Quarter'] = df['Date'].dt.quarter

In [ ]:
df.rename(columns={ # RCFD variables (Balance Sheet):
                'RCFD2170':'Total Assets', 'RCFD1400': 'Loans', 'RCFD2200': 'Deposits', 
                'RCFD1403': 'Loans Non-Accrual', 'RCFD1407':'Loans Past Due 90 Days',
                'RCFD0010': 'Cash', 
                # RIAD variables (Income Statement):
                'RIAD4170':'Interest Exp. Deposits', 
                'RIAD4180': 'Interest Exp. FedFunds', 'RIAD4635': 'Charge Off Loans', 
                'RIAD4605': 'Recovery Loans', 'RIAD4130': 'Total Expenses', 
                'RIAD4217': 'Expenses on premises and fixed assets', 
                'RIAD4135': 'Labor expenses', 'RIAD4079': 'Total Non-interest Income'}, inplace=True)

In [ ]:
df['Interest Income Loans'] = df['RIAD4010'].fillna(0)+df['RIAD4065'].fillna(0)
df['Dividends'] = df['RIAD4470'].fillna(0)+df['RIAD4460'].fillna(0)

In [ ]:
df['Federal Funds Purchased'] = np.where(df['Date'].dt.year < 2002, df['RCFD2800'], df['RCFDB993']+df['RCFDB995'])
df['Tier 1 Capital'] = np.where(df['Date'].dt.year < 2014, df['RCFD8274'], df['RCFA8274'])
df['Risk-Weighted Assets'] = np.where(df['Date'].dt.year < 2014, df['RCFDA223'], df['RCFAA223'])
df['Other Borrowings'] = np.where(df['Date'].dt.year < 2001, df['RCFD2835'], df['RCFD3190'])
df['FedFunds Sold'] = np.where(df['Date'].dt.year < 2002, df['RCFD1350'], df['RCFDB987']+df['RCFDB989'])
df['US Treasury Securities'] = np.where(df['Date'].dt.year < 1994, df['RCFD0400'], df['RCFD0211']+df['RCFD1287'])

In [ ]:
# For 'US Agency Obligations' we need a np.where with 3 time windows (1984-1993, 1994-2008, 2009-2010, 2011-2021):
df['US Agency Obligations'] = np.where(
                            df['Date'].dt.year < 1994, df['RCFD0600'],
                            np.where(
                            df['Date'].dt.year < 2009, 
                            df['RCFD1289'].fillna(0)+df['RCFD1294'].fillna(0)+df['RCFD1293'].fillna(0)+df['RCFD1298'].fillna(0)+
                            df['RCFD1698'].fillna(0)+df['RCFD1702'].fillna(0)+df['RCFD1703'].fillna(0)+df['RCFD1707'].fillna(0)+
                            df['RCFD1714'].fillna(0)+df['RCFD1717'].fillna(0)+df['RCFD1718'].fillna(0)+df['RCFD1732'].fillna(0),
                            np.where(
                            df['Date'].dt.year < 2011, 
                            df['RCFD1289'].fillna(0)+df['RCFD1294'].fillna(0)+df['RCFD1293'].fillna(0)+df['RCFD1298'].fillna(0)+
                            df['RCFDG300'].fillna(0)+df['RCFDG303'].fillna(0)+df['RCFDG304'].fillna(0)+df['RCFDG307'].fillna(0)+
                            df['RCFDG312'].fillna(0)+df['RCFDG315'].fillna(0)+df['RCFDG316'].fillna(0)+df['RCFDG319'].fillna(0)+
                            df['RCFDG324'].fillna(0)+df['RCFDG327'].fillna(0),
                            df['RCFD1289'].fillna(0)+df['RCFD1294'].fillna(0)+df['RCFD1293'].fillna(0)+df['RCFD1298'].fillna(0)+
                            df['RCFDG300'].fillna(0)+df['RCFDG303'].fillna(0)+df['RCFDG304'].fillna(0)+df['RCFDG307'].fillna(0)+
                            df['RCFDG312'].fillna(0)+df['RCFDG315'].fillna(0)+df['RCFDG316'].fillna(0)+df['RCFDG319'].fillna(0) 
# Missing here RCFDK142 and RCFDK145.
                            )
                            )
)

In [ ]:
df['Interest Income Safe Securities'] = np.where(df['Date'].dt.year<2001, df['RIAD4027'], df['RIADB488'])
df['Equity Issuance'] = np.where(df['Date'].dt.year<2001, df['RIAD4346'].fillna(0)+df['RIADB510'].fillna(0), 
                                 df['RIADB509'].fillna(0)+df['RIADB510'].fillna(0))

In [ ]:
# Derived variables:
df['Loans (RW Assets) to Assets'] = df['Risk-Weighted Assets']/df['Total Assets']
df['Cash and Securities to Assets'] = 1 - df['Loans (RW Assets) to Assets']
df['Capital Ratio (risk-weighted)'] = df['Tier 1 Capital']/df['Risk-Weighted Assets']
df['Equity to Assets'] = df['Tier 1 Capital']/df['Total Assets']
df['Deposits to Assets'] = 1 - df['Equity to Assets']
df['Interest Return on Loans'] = df['Interest Income Loans']/df['Loans']
df['Interest Cost Deposits'] = df['Interest Exp. Deposits']/df['Deposits']
df['Loan Interest Margin'] = df['Interest Income Loans']-df['Interest Exp. Deposits']
df['Cost Fed Funds'] = df['Interest Exp. FedFunds']/df['Federal Funds Purchased']
df['Charge Off Rate Loans']  = (df['Charge Off Loans']-df['Recovery Loans'])/df['Loans']
df['Delinquency Rate Loans'] = (df['Loans Non-Accrual']+df['Loans Past Due 90 Days'])/df['Loans']
df['Safe Securities'] = df['US Treasury Securities']+df['US Agency Obligations']
df['Cost of Funds'] = (df['Interest Exp. Deposits']+df['Interest Exp. FedFunds'])/(df['Deposits']+df['Federal Funds Purchased'])
df['Interest Return on Safe Assets'] = df['Interest Income Safe Securities'] / df['Safe Securities']
df['Return Safe Securities'] = df['Interest Return on Safe Assets'] - df['Cost of Funds']
df['Return on Loans'] = df['Interest Return on Loans'] - df['Charge Off Rate Loans']

In [ ]:
# keep only the variables we created/renamed:
df = df[['Date', 'Year', 'Quarter', 'Financial Institution Name', 'IDRSSD', 'Total Assets', 'Loans', 'Deposits', 
         'Loans Non-Accrual', 'Loans Past Due 90 Days', 'Cash', 'Interest Income Loans', 'Dividends', 
         'Interest Exp. Deposits', 'Interest Exp. FedFunds', 'Charge Off Loans', 'Recovery Loans', 
         'Total Expenses', 'Expenses on premises and fixed assets', 'Labor expenses', 'Total Non-interest Income', 
         'Federal Funds Purchased', 'Tier 1 Capital', 'Risk-Weighted Assets', 'Other Borrowings', 'FedFunds Sold', 
         'US Treasury Securities', 'US Agency Obligations', 'Interest Income Safe Securities', 'Equity Issuance', 
         'Loans (RW Assets) to Assets', 'Cash and Securities to Assets', 'Capital Ratio (risk-weighted)', 
         'Equity to Assets', 'Deposits to Assets', 'Interest Return on Loans', 'Interest Cost Deposits', 
         'Loan Interest Margin', 'Cost Fed Funds', 'Charge Off Rate Loans', 'Delinquency Rate Loans', 
         'Safe Securities', 'Cost of Funds', 'Interest Return on Safe Assets', 'Return Safe Securities', 
         'Return on Loans']]

Recall that plenty of the variables computed here are ratios. So if some banks have a zero for the denominator, those ratios would explode. 

## M&A Merges

In [ ]:
df2 = df.copy()

In [ ]:
transformations = pd.read_csv('C:/Users/angel/Documents/Economics/Research/Banking Project/data/clean/CSV_TRANSFORMATIONS.csv')

transformations['DT_TRANS'] = transformations['DT_TRANS'].astype(str)

transformations['Year'] = transformations['DT_TRANS'].str[:4]
transformations['Month'] = transformations['DT_TRANS'].str[4:6]
transformations['Day'] = transformations['DT_TRANS'].str[6:8]
transformations['Transaction Date'] = pd.to_datetime(transformations['Year'] + '-' + transformations['Month'] + '-' + transformations['Day'])

# drop the Month, Day, and Year columns:
transformations.drop(['Year', 'Month', 'Day'], axis=1, inplace=True)

# Rename #ID_RSSD_PREDECESSOR to IDRSSD Predecessor:
transformations.rename(columns={'#ID_RSSD_PREDECESSOR':'IDRSSD Predecessor',
                                'ID_RSSD_SUCCESSOR': 'IDRSSD Successor', 
                                'TRNSFM_CD': 'Transformation Code'}, inplace=True)

# make those columns integers:
transformations['IDRSSD Predecessor'] = transformations['IDRSSD Predecessor'].astype(int)
transformations['IDRSSD Successor'] = transformations['IDRSSD Successor'].astype(int)

transformations = transformations[['Transaction Date', 'IDRSSD Predecessor', 'IDRSSD Successor', 'Transformation Code']]


In [ ]:
df_predecessor = pd.merge(df2, transformations, left_on='IDRSSD', right_on='IDRSSD Predecessor', how='left')
df_successor = pd.merge(df2, transformations, left_on='IDRSSD', right_on='IDRSSD Successor', how='left')